In [22]:
import torch
from LayoutDM import CLDM

In [23]:
dict =torch.load('/workspace/joonsm/City_Layout/log_dir/baseline_512/checkpoints/checkpoint-150/pytorch_model.bin')

In [24]:
a = CLDM(use_temp=True)

In [31]:
a.transformer.encoder.layers[0]

tempcoder(
  (self_attention): MultiHeadAttentionLayer(
    (q_fc): Linear(in_features=256, out_features=256, bias=True)
    (k_fc): Linear(in_features=256, out_features=256, bias=True)
    (v_fc): Linear(in_features=256, out_features=256, bias=True)
    (out_fc): Linear(in_features=256, out_features=256, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (temp_attention): MultiHeadAttentionLayer(
    (q_fc): Linear(in_features=256, out_features=256, bias=True)
    (k_fc): Linear(in_features=256, out_features=256, bias=True)
    (v_fc): Linear(in_features=256, out_features=256, bias=True)
    (out_fc): Linear(in_features=256, out_features=256, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (position_ff): PositionWiseFeedForwardLayer(
    (fc1): Linear(in_features=256, out_features=512, bias=True)
    (relu): ReLU()
    (dropout): Dropout(p=0.0, inplace=False)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
  )
  (residual1): ResidualConnectionL

In [19]:
a =torch.rand(1,2)

In [21]:


a.size()

torch.Size([1, 2])